# 📊 Tech Challenge - Análise e Previsão de Fatores de Estresse em Estudantes

## 📑 Sumário
1. [📝 Definição do problema](#1-📝-Definição-do-problema)
2. [📥 Coleta e preparação dos dados](#2-📥-Coleta-e-preparação-dos-dados)
3. [🔎 Análise exploratória de dados (EDA)](#3-🔎-Análise-exploratória-de-dados-EDA)
4. [⚙️ Engenharia e seleção de atributos](#4-⚙️-Engenharia-e-seleção-de-atributos)
5. [🤖 Modelagem](#5-🤖-Modelagem)
6. [📏 Avaliação e interpretação dos resultados](#6-📏-Avaliação-e-interpretação-dos-resultados)
7. [🚀 Deploy](#7-🚀-Deploy)

## 1. 📝 Definição do problema

* **Problema de Negócio:** Desenvolver uma solução que permita a instituições de ensino identificar estudantes com risco de estresse elevado, possibilitando a criação de programas de apoio mais direcionados e eficazes.
* **Objetivo Técnico:** O problema foi enquadrado como um desafio de **Classificação Multiclasse**. O objetivo é treinar um modelo de Machine Learning para prever o nível de estresse de um estudante em uma de três categorias (0: baixo, 1: médio, 2: alto), utilizando a coluna `stress_level` como variável-alvo.

## 2. 📥 Coleta e preparação dos dados

#### 📌 Coleta de Dados

* **Fonte dos Dados:** Foi utilizado um dataset público da plataforma Kaggle, chamado "Student Stress Factors - A Comprehensive Analysis".
* **Justificativa:** A utilização de um dataset já existente foi uma decisão estratégica para otimizar o tempo e concentrar os esforços do projeto nas etapas de Análise, Modelagem e Deploy, que são o foco do desafio.

#### 📌 Armazenamento dos Dados

* **Solução Adotada:** Para simular um ambiente de produção e garantir a reprodutibilidade, os dados foram armazenados em um sistema de armazenamento de objetos local (MinIO)].
* **Processo:** Um bucket chamado `student-stress` foi criado no servidor MinIO , e o dataset `StressLevelDataset.csv` foi carregado, tornando-se acessível via serviço para a etapa de análise.

In [ ]:
# Importação de Bibliotecas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import boto3
from io import BytesIO

# Configurações de estilo para os gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Configurações de conexão com o MinIO
MINIO_ENDPOINT = "127.0.0.1:9000"
MINIO_ACCESS_KEY = "minioadmin"  # Use o seu usuário
MINIO_SECRET_KEY = "minioadmin"  # Use a sua senha
BUCKET_NAME = "student-stress"
OBJECT_NAME = "StressLevelDataset.csv"

# Criar um cliente S3 compatível para interagir com o MinIO
s3_client = boto3.client(
    "s3",
    endpoint_url=f"http://{MINIO_ENDPOINT}",
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY,
    aws_session_token=None,
    config=boto3.session.Config(signature_version="s3v4"),
)

# Tentar carregar o objeto do bucket
try:
    response = s3_client.get_object(Bucket=BUCKET_NAME, Key=OBJECT_NAME)
    # Ler o conteúdo do objeto em um DataFrame do Pandas
    # O BytesIO trata os bytes do objeto como um arquivo em memória
    df = pd.read_csv(BytesIO(response["Body"].read()))
    print("Dataset carregado com sucesso do MinIO!")
except Exception as e:
    print(f"Erro ao carregar o dataset do MinIO: {e}")

#### 📌 Inspeção Inicial do DataFrame

In [ ]:
# Visualizar as 5 primeiras linhas do dataset
print("Primeiras 5 linhas do dataset:")
display(df.head())

# Obter informações gerais sobre o DataFrame (tipos de dados, valores não nulos)
print("\nInformações gerais do DataFrame:")
df.info()

# Obter estatísticas descritivas para as colunas numéricas
print("\nEstatísticas Descritivas:")
display(df.describe())

# Verificar as dimensões do dataset (linhas, colunas)
print(f"\nO dataset possui {df.shape[0]} linhas e {df.shape[1]} colunas.")

# Contar o número de linhas completamente duplicadas no DataFrame
num_duplicates = df.duplicated().sum()
print(f"Número de linhas duplicadas encontradas: {num_duplicates}")

#### Análise Preliminar e Qualidade dos Dados

A inspeção inicial do dataset, realizada através dos comandos `.info()`, `.describe()` e `.head()`, revelou os seguintes insights cruciais que guiarão as próximas etapas:

* **Ausência de Dados Faltantes:** Todas as 21 colunas contêm 1100 registros não nulos. Isso indica uma excelente qualidade de dados e nos isenta da necessidade de aplicar técnicas de tratamento de valores ausentes (imputação).

* **Formato Numérico Uniforme:** Todas as variáveis são do tipo `int64`, o que simplifica a etapa de pré-processamento, pois não será necessário realizar a conversão de tipos de dados (ex: *encoding* de variáveis categóricas textuais).

* **Natureza Implícita das Variáveis:** Apesar do tipo `int64`, a análise semântica e estatística (`min`, `max`) sugere que as colunas representam diferentes tipos de dados. Temos uma mistura de variáveis que, na prática, são categóricas binárias (ex: `mental_health_history`), ordinais (escalas de 0 a 5) e a variável-alvo multiclasse (`stress_level`), que é o foco da nossa previsão.

------

#### Conclusão sobre a Variável-Alvo

A análise da distribuição da variável `stress_level` mostra que as três classes (0: Baixo, 1: Médio, 2: Alto) estão **muito bem balanceadas**. Com contagens de 373, 358 e 369 respectivamente, temos uma base sólida e imparcial para o treinamento do modelo de classificação, eliminando a necessidade de técnicas de balanceamento de classes.

#### Resumo da Análise de Qualidade dos Dados

A análise de qualidade, que incluiu a verificação de valores ausentes e registros duplicados, revelou um dataset excepcionalmente limpo. As principais conclusões são:

* **Dados Completos:** Não foi encontrado **nenhum valor ausente** em nenhuma das 21 colunas.
* **Dados Únicos:** A verificação confirmou que não há **nenhuma linha duplicada** no conjunto de dados.
* **Formato Adequado:** Todas as variáveis já estão no formato numérico (`int64`), o que simplifica as etapas futuras de pré-processamento.

Com base nisso, a fase de limpeza de dados é considerada concluída, e podemos prosseguir com confiança para a análise de distribuição das variáveis.

#### 5. Análise da Variável-Alvo (stress_level)

In [ ]:
# --- Análise Univariada ---

# 1. Análise da Variável-Alvo: stress_level
plt.figure(figsize=(8, 6))
sns.countplot(x="stress_level", data=df, palette="viridis")

# Adicionando títulos e rótulos para maior clareza
plt.title("Distribuição da Variável-Alvo (Nível de Estresse)", fontsize=16)
plt.xlabel("Nível de Estresse (0: Baixo, 1: Médio, 2: Alto)", fontsize=12)
plt.ylabel("Contagem de Estudantes", fontsize=12)

# Exibindo o gráfico
plt.show()

# Para complementar o gráfico, vamos exibir a contagem exata de cada classe
print("Contagem de valores para cada nível de estresse:")
print(df["stress_level"].value_counts())

#### 6. Histogramas de Todas as Variáveis Preditivas

In [ ]:
# 2. Análise das Variáveis Preditivas (Features)

# Selecionar todas as colunas, exceto a nossa variável-alvo 'stress_level'
features = df.drop("stress_level", axis=1).columns

# Criar um histograma para cada feature
for feature in features:
    plt.figure(figsize=(10, 5))
    # Usaremos um histograma, ideal para variáveis numéricas/ordinais
    sns.histplot(
        df[feature], kde=True, bins=20
    )  # kde=True adiciona uma linha de densidade
    plt.title(f"Distribuição de: {feature}", fontsize=15)
    plt.xlabel(feature, fontsize=12)
    plt.ylabel("Frequência", fontsize=12)
    plt.show()